In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
SEED = int(time.time())

import sys
sys.path.append('..')

from os.path import join
from datetime import datetime

DATAFOLDER = join('..', 'data')

# Load data and get chunks

In [2]:
data = pd.read_csv(join(DATAFOLDER, 'matches-Brazil.csv'))
data.head()

,league,area,retrieved_from_url,start_time,end_time,team1,team2,team1_score,team2_score,outcome,team1_odds,team2_odds,draw_odds
0,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,1559523600,1559529000,Atletico-MG,CSA,4.0,0.0,TEAM1,1.41,7.98,4.31
1,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,1559523600,1559529000,Chapecoense-SC,Palmeiras,1.0,2.0,TEAM2,5.92,1.63,3.51
2,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,1559523600,1559529000,Internacional,Avai,2.0,0.0,TEAM1,1.43,8.22,4.15
3,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,1559512800,1559518200,Athletico-PR,Fluminense,3.0,0.0,TEAM1,1.64,5.39,3.77
4,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,1559512800,1559518200,Ceara,Santos,0.0,1.0,TEAM2,2.69,2.65,3.16


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6412 entries, 0 to 6411
Data columns (total 13 columns):
league                6412 non-null object
area                  6412 non-null object
retrieved_from_url    6412 non-null object
start_time            6412 non-null int64
end_time              6412 non-null int64
team1                 6412 non-null object
team2                 6412 non-null object
team1_score           6411 non-null float64
team2_score           6411 non-null float64
outcome               6412 non-null object
team1_odds            6412 non-null object
team2_odds            6412 non-null object
draw_odds             6412 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 651.3+ KB


Check for NaN entries

In [4]:
data_dropped = data.dropna()
data_dropped.shape

(6411, 13)

Check for duplicates

In [5]:
data_dropped = data_dropped.drop(data_dropped.loc[data_dropped.duplicated()].index)
data_dropped.shape

(6031, 13)

# UTC data to readable data

In [6]:
def utc2time(d):
    return datetime.utcfromtimestamp(d).strftime('%Y-%m-%d %H:%M:%S')

In [7]:
data.start_time = data_dropped.start_time.apply(utc2time)
data.end_time = data_dropped.end_time.apply(utc2time)

In [8]:
type(data.start_time[0])

str

### Sort data

In [9]:
data.sort_values(by='start_time', inplace=True)
data.head()

,league,area,retrieved_from_url,start_time,end_time,team1,team2,team1_score,team2_score,outcome,team1_odds,team2_odds,draw_odds
6031,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2004-04-21 22:00:00,2004-04-21 23:30:00,Sao Caetano,Vitoria,1.0,0.0,TEAM1,1.5,6.17,3.48
6030,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2004-04-21 22:00:00,2004-04-21 23:30:00,Figueirense,Internacional,1.0,0.0,TEAM1,2.1,3.15,3.17
6029,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2004-04-21 22:00:00,2004-04-21 23:30:00,Botafogo RJ,Goias,1.0,4.0,TEAM2,1.9,3.64,3.2
6028,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2004-04-22 00:00:00,2004-04-22 01:30:00,Palmeiras,Atletico-MG,0.0,0.0,DRAW,1.95,3.6,3.25
6027,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2004-04-22 00:00:00,2004-04-22 01:30:00,Cruzeiro,Juventude,2.0,1.0,TEAM1,1.33,-,4.18


In [10]:
idx = data['start_time'] < '2018'
data_until2018 = data.loc[idx]
data_from2018 = data.drop(data_until2018.index)

In [11]:
data_from2018.shape

(830, 13)

In [12]:
data_until2018.shape

(5582, 13)

In [13]:
data.shape

(6412, 13)

# Overall data analyses

Let us take a look how many teams are in this database

In [14]:
teams1 = data_until2018['team1'].unique()
teams2 = data_until2018['team2'].unique()
all_teams = np.append(teams1, teams2)
all_teams

array(['Sao Caetano', 'Figueirense', 'Botafogo RJ', 'Palmeiras',
       'Cruzeiro', 'Ponte Preta', 'Gremio', 'Coritiba', 'Vasco',
       'Sao Paulo', 'Paysandu PA', 'Goias', 'Flamengo RJ', 'Juventude',
       'Internacional', 'Guarani', 'Fluminense', 'Corinthians',
       'Athletico-PR', 'Vitoria', 'Santos', 'Criciuma', 'Parana',
       'Atletico-MG', 'Fortaleza', 'Brasiliense', 'Santa Cruz',
       'Sport Recife', 'America RN', 'Nautico', 'Betim', 'Portuguesa',
       'Avai', 'Santo Andre', 'Barueri', 'Atletico GO', 'Ceara',
       'America MG', 'Bahia', 'Chapecoense-SC', 'Joinville', 'Vitoria',
       'Internacional', 'Goias', 'Atletico-MG', 'Juventude',
       'Corinthians', 'Flamengo RJ', 'Guarani', 'Criciuma',
       'Athletico-PR', 'Fluminense', 'Cruzeiro', 'Ponte Preta', 'Gremio',
       'Palmeiras', 'Sao Caetano', 'Coritiba', 'Paysandu PA',
       'Figueirense', 'Parana', 'Botafogo RJ', 'Sao Paulo', 'Santos',
       'Vasco', 'Fortaleza', 'Brasiliense', 'Santa Cruz', 'Nautico',


In [15]:
all_teams = np.unique(all_teams)
all_teams

array(['America MG', 'America RN', 'Athletico-PR', 'Atletico GO',
       'Atletico-MG', 'Avai', 'Bahia', 'Barueri', 'Betim', 'Botafogo RJ',
       'Brasiliense', 'Ceara', 'Chapecoense-SC', 'Chapecoense-SC\xa0',
       'Corinthians', 'Coritiba', 'Criciuma', 'Cruzeiro', 'Figueirense',
       'Flamengo RJ', 'Fluminense', 'Fortaleza', 'Goias', 'Gremio',
       'Gremio\xa0', 'Guarani', 'Internacional', 'Joinville', 'Juventude',
       'Nautico', 'Palmeiras', 'Parana', 'Paysandu PA', 'Ponte Preta',
       'Portuguesa', 'Santa Cruz', 'Santo Andre', 'Santos', 'Sao Caetano',
       'Sao Paulo', 'Sport Recife', 'Vasco', 'Vitoria', 'Vitoria\xa0'],
      dtype=object)

In [16]:
all_teams.shape

(44,)

There are 24 teams that participate in league up to 2018. Let's take a look on:
- How many matches which teams has been participated.

In [17]:
def set_year(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return date.year

In [18]:
data_until2018['year'] = data_until2018['start_time'].apply(set_year)

/home/navar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
data_team_by_year = data_until2018.groupby(by=['year', 'team1']).team1.count()

In [20]:
date

NameError: name 'date' is not defined

In [ ]:
data_team_by_year.head()

In [ ]:
data_team_by_year.loc[2004].plot()

In a give year all team attent to the same number o matches.

In [ ]:
gb_year = data_until2018.groupby(by='year')
gb_year.get_group(2010).head()

In [ ]:
count_outcome_by_year = gb_year.outcome.value_counts().unstack()
count_outcome_by_year

In [ ]:
count_outcome_by_year.plot(kind='barh', stacked=True, figsize=[16,6], colormap='winter')

We can see that the number of matches that resulted in `DRAW` tend to be the same over a 12 year period. 

In [ ]:
count_outcome_by_year.describe()

In [ ]:
count_outcome_by_year.describe()

In [ ]:
def matches_percentage(row):
    return row / row.sum()

In [ ]:
for idx, row in count_outcome_by_year.iterrows():
    count_outcome_by_year.loc[idx] = matches_percentage(row)

In [ ]:
count_outcome_by_year.describe()

About **26%** of matches resulted in `DRAW`

But what happened in **2010**?

In [ ]:
(gb_year.outcome.count() / gb_year.outcome.count().max()).plot()

In 2004 there were more about **30%** more matches than in other years. Why is that? Is that an outlier? Let us to investigate:
- How many teams are participating each season
- How many matches teams have attend in average.
- Compare everything agains 2010's data. (There were some duplicates. fixed at the beggining)

In [ ]:
gb_year.team1.count()

In [ ]:
number_teams = []
number_of_matches = []
average_matches = []
year = []

for idx, row in gb_year.team1.unique().items():
    average_matches.append(gb_year.team1.count()[idx] / len(row))
    number_teams.append(len(row))
    number_of_matches.append(gb_year.team1.count()[idx])
    year.append(idx)
    
teams = {'number_of_matches': number_of_matches,
         'number_teams': number_teams,
         'average_matches': average_matches}

In [ ]:
df_teams = pd.DataFrame(data=teams, index=year)
df_teams

In [ ]:
df_teams.plot(subplots=True, layout=(1,3), figsize=(16,6))
plt.show()

There were more team participating the league back in 2004.

# Teams statistics

In [ ]:
gb_team1 = data_until2018.groupby(by='team1')

In [ ]:
gb_team1.year.value_counts().head()

## How the results are distributed

In [ ]:
outcome_by_team = gb_team1.outcome.value_counts().unstack()

In [ ]:
outcome_by_team.plot(kind='bar', stacked=True, figsize=[16,5], colormap='copper')
plt.show()

Let us answer:
- How many matches each team has player and how they are distributed over the period.

In [ ]:
outcome_by_team['matches'] = outcome_by_team.sum(axis=1)
outcome_by_team.head()

In [ ]:
outcome_by_team['matches'].plot(kind='bar', stacked=True, figsize=[16,5], colormap='copper')
plt.show()

In [ ]:
gb_outcome = data_until2018.groupby(by=['outcome', 'year'])
gb_outcome

In [ ]:
gb_outcome['team1']

In [ ]:
gb_team1 = data_until2018.groupby(by='team1')
gb_year.get_group('Palmeiras').head()

In [ ]:
data_from2018.groupby('start_time')

In [ ]:
data_from2018.groupby('start_time')

# Export data

In [ ]:
data_until2018.to_csv(join(DATAFOLDER, 'matches-Brazil-2004-2017.csv'),index=False)

In [128]:
u = data_until2018.groupby('year')

In [138]:
years_range = np.array([2004, 2005, 2006, 2007])

def split():
    for y in range(max(years_range.shape)-1):
        print("it {}".format(y))
        print(years_range[:y+1])
        print(years_range[y+1:])

        years_train = years_range[:y+1]
        years_test = years_range[y+1:]

        #train_idx = []
        train_idx = np.array([])
        for year in years_train:
            train_idx = np.append(train_idx, u.indices[year])
            #train_idx.append(u.indices[year])
            
        yield train_idx
        
    
    

In [140]:
ui = []
for i in split():
    ui.append(data_until2018.iloc[i])
    #print(i)

it 0
[2004]
[2005 2006 2007]
it 1
[2004 2005]
[2006 2007]
it 2
[2004 2005 2006]
[2007]


In [149]:
ui[1].shape

(997, 14)

In [116]:
data_until2018.iloc[i].tail()

,league,area,retrieved_from_url,start_time,end_time,team1,team2,team1_score,team2_score,outcome,team1_odds,team2_odds,draw_odds,year
4660,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2006-12-03 21:00:00,2006-12-03 22:30:00,Figueirense,Vasco,0.0,0.0,DRAW,2.22,2.8,3.24,2006
4659,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2006-12-03 21:00:00,2006-12-03 22:30:00,Cruzeiro,Botafogo RJ,3.0,1.0,TEAM1,1.63,4.77,3.38,2006
4658,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2006-12-03 21:00:00,2006-12-03 22:30:00,Corinthians,Juventude,5.0,3.0,TEAM1,1.55,5.0,3.6,2006
4657,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2006-12-03 23:10:00,2006-12-04 00:40:00,Santos,Santa Cruz,3.0,1.0,TEAM1,1.18,11.17,5.42,2006
4656,Serie A,Brazil,https://www.oddsportal.com/soccer/brazil/serie...,2006-12-03 23:10:00,2006-12-04 00:40:00,Fortaleza,Gremio,1.0,0.0,TEAM1,4.8,1.65,3.25,2006


In [ ]:
u = [1,2,3,4]

In [ ]:
from sklearn.model_selection import KFold

kf = KFold()

In [ ]:
for a, b in kf.split(u):
    print(a)
    print(b)

In [67]:
=

TypeError: list indices must be integers or slices, not list